In [3]:
!pip install openai==0.28.1
!pip install tiktoken==0.6.0
!pip install langchain==0.1.20

In [4]:
import os
import openai
import numpy as np
import pandas as pd
import json
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from openai.embeddings_utils import get_embedding

In [5]:
openai.api_key = ""

In [6]:
os.environ["OPENAI_API_KEY"] = ""

In [10]:
df = pd.read_csv('https://raw.githubusercontent.com/AI-Republic-PH/AIR_AI_Engineering_Course_2024/refs/heads/main/Datasets/IMDB_Dataset.csv', nrows=20)

In [11]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [12]:
df['review'][0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [27]:
System_Prompt = """
You are an AI assistant trained to perform sentiment analysis on individual movie reviews from the IMDB Dataset. For each review provided, your task is to determine the overall sentiment—either 'Positive' or 'Negative'—based on the dominant tone, language, and context within the text. This decision will consider the general feeling conveyed by the reviewer rather than isolated phrases.

Instructions for Analyzing Reviews:
Primary Objective: Classify each review into one of two categories—'Positive' or 'Negative'—based on whether the review predominantly praises or criticizes the movie.

Response Format:

Respond with exactly one word: 'Positive' if the review is overall favorable.
Respond with exactly one word: 'Negative' if the review is overall unfavorable.
Classification Guidelines:

Positive:
Choose 'Positive' if the review contains predominantly positive language, enthusiastic comments, or praise, even if minor criticisms are present.
Reviews expressing satisfaction with the movie's core elements, like storyline, acting, visuals, or emotional impact, should be classified as 'Positive.'
If the review is mixed but leans more toward favorable impressions, select 'Positive.'
Negative:
Choose 'Negative' if the review contains predominantly negative language, expresses disappointment, or criticizes key aspects of the movie.
Reviews that mention dissatisfaction, boredom, lack of coherence, poor acting, or weak storyline should be classified as 'Negative.'
If the review is mixed but overall conveys a sense of disappointment or unfavorable impression, select 'Negative.'
Additional Considerations:

Ambiguity: If a review is unclear or neutral but leans slightly toward one sentiment, choose the sentiment that aligns most closely with the general tone.
Mixed Reviews: For reviews with both positive and negative aspects, focus on the dominant sentiment. If the positive comments outweigh the negative, classify it as 'Positive,' and vice versa.
Restrictions:

Do not provide any additional text, explanations, or analysis beyond 'Positive' or 'Negative.'
Refrain from asking clarifying questions, even if the sentiment appears ambiguous.
Avoid adding qualifiers or subjective interpretations; adhere strictly to the two-category classification.
Example Responses:
Review: "The movie had a few slow parts but overall was really engaging and well-acted."
Response: Positive

Review: "The plot was confusing and the acting was mediocre, though some scenes were visually nice."
Response: Negative

By adhering to these guidelines, you will provide a single-word classification for each movie review, capturing the essence of the reviewer’s sentiment in a straightforward and consistent way.
"""

In [28]:
struct = [{'role' : 'system', 'content' : System_Prompt}]

user_message = df['review'][3]
print("User:", user_message)
struct.append({'role' : 'system', 'content' : user_message})

chat = openai.ChatCompletion.create(model="gpt-4o-mini", messages = struct)
response = chat.choices[0].message.content

struct.append({"role" : "assistant", "content" : response})
print("Assistant : " + response)

User: Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenly, Jake decides to become Rambo and kill the zombie.<br /><br />OK, first of all when you're going to make a film you must Decide if its a thriller or a drama! As a drama the movie is watchable. Parents are divorcing & arguing like in real life. And then we have Jake with his closet which totally ruins all the film! I expected to see a BOOGEYMAN similar movie, and instead i watched a drama with some meaningless thriller spots.<br /><br />3 out of 10 just for the well playing parents & descent dialogs. As for the shots with Jake: just ignore them.
Assistant : Negative


In [29]:
df['Openai_Sentiment'] = ''

In [30]:
for x in range(0, len(df)):
    struct = [{'role' : 'system', 'content' : System_Prompt}]
    user_message = df['review'][x]
    print("User:", user_message)
    struct.append({'role' : 'system', 'content' : user_message})

    chat = openai.ChatCompletion.create(model="gpt-4o-mini", messages = struct)
    response = chat.choices[0].message.content
    print("Assistant : " + response)
    struct.append({"role" : "assistant", "content" : response})
    df['Openai_Sentiment'][x] = response.lower()

User: One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to t

<ipython-input-30-a196d8e3ac83>:11: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['Openai_Sentiment'][x] = response.lower()


Assistant : Positive
User: I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). While some may be disappointed when they realize this is not Match Point 2: Risk Addiction, I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to love.<br /><br />This was the most I'd laughed at one of Woody's comedies in years (dare I say a decade?). While I've never been impressed with Scarlet Johanson, in this she managed to tone down her "sexy" image and jumped right into a average, but spirited young woman.<br /><br />This may not be the crown jewel of his career, but it was wittier than "Devil Wears Prada" and more interesting than "Superman" a great comedy to go see with friends.


<ipython-input-30-a196d8e3ac83>:11: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['Openai_Sentiment'][x] = response.lower()


Assistant : Positive
User: Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenly, Jake decides to become Rambo and kill the zombie.<br /><br />OK, first of all when you're going to make a film you must Decide if its a thriller or a drama! As a drama the movie is watchable. Parents are divorcing & arguing like in real life. And then we have Jake with his closet which totally ruins all the film! I expected to see a BOOGEYMAN similar movie, and instead i watched a drama with some meaningless thriller spots.<br /><br />3 out of 10 just for the well playing parents & descent dialogs. As for the shots with Jake: just ignore them.


...

<ipython-input-30-a196d8e3ac83>:11: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['Openai_Sentiment'][x] = response.lower()


Assistant : Positive
User: An awful film! It must have been up against some real stinkers to be nominated for the Golden Globe. They've taken the story of the first famous female Renaissance painter and mangled it beyond recognition. My complaint is not that they've taken liberties with the facts; if the story were good, that would perfectly fine. But it's simply bizarre -- by all accounts the true story of this artist would have made for a far better film, so why did they come up with this dishwater-dull script? I suppose there weren't enough naked people in the factual version. It's hurriedly capped off in the end with a summary of the artist's life -- we could have saved ourselves a couple of hours if they'd favored the rest of the film with same brevity.


<ipython-input-30-a196d8e3ac83>:11: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['Openai_Sentiment'][x] = response.lower()


Assistant : Negative


<ipython-input-30-a196d8e3ac83>:11: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['Openai_Sentiment'][x] = response.lower()


In [31]:
df.head(20)

,review,sentiment,Openai_Sentiment
0,One of the other reviewers has mentioned that ...,positive,positive
1,A wonderful little production. <br /><br />The...,positive,positive
2,I thought this was a wonderful way to spend ti...,positive,positive
3,Basically there's a family where a little boy ...,negative,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,positive
5,"Probably my all-time favorite movie, a story o...",positive,positive
6,I sure would like to see a resurrection of a u...,positive,positive
7,"This show was an amazing, fresh & innovative i...",negative,negative
8,Encouraged by the positive comments about this...,negative,negative
9,If you like original gut wrenching laughter yo...,positive,positive


In [32]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(df['sentiment'], df['Openai_Sentiment'])
print(f"Accuracy: {accuracy}")

Accuracy: 0.95
